In [2]:
import pandas as pd
import random

df = pd.read_csv('data/data.csv')

df[:3]

,client_id,gender,birth_date,create_date,nonresident_flag,businessman_flag,city,term,contract_sum,product_category_name,...,card_type_name,start_date,fact_close_date,purchase_sum,purchase_count,current_balance_avg_sum,current_balance_sum,current_debit_turn_sum,current_credit_turn_sum,card_type
0,fe60b594364f9f636266ed1ef4f89c32,Ж,1985,2020-01-29,R,0,Гусь-Хрустальный,NaN,0.0,Договор на текущий счет для дебетовой карты,...,Visa Platinum Rewards,2019.0,2021.0,16600.0,71,39700.0,25700.0,220600.0,201000.0,dc
1,3012cabca5885ed53d348d6e57dab5de,М,1951,2009-09-24,R,0,Ишимбай,NaN,0.0,Договор на текущий счет для дебетовой карты,...,VISA Classic,2019.0,NaN,0.0,0,1900.0,1000.0,6200.0,3000.0,dc
2,d871ef96820b6c7a1ada8e01a772724e,Ж,1982,2006-12-25,R,0,Юрга,NaN,0.0,Договор на текущий счет для дебетовой карты,...,NaN,NaN,NaN,0.0,0,0.0,0.0,0.0,0.0,dc


In [5]:
regions = pd.read_csv('data/regions.csv', sep=';')

regions_grp = regions[['settlement','oktmo']].groupby('oktmo').last()
regions_grp.reset_index().rename(columns={"oktmo": "id_region", "settlement": "region"}).to_json(r'data/regions.json', orient='records', force_ascii=False)

regions.head()

,region,municipality,settlement,oktmo,latitude_dd,longitude_dd,year,assets,assets_depreciation,assets_exhausted,...,retail_growth,rni,schools,servises,volume_electr,volume_manufact,volume_mining,volume_water,wage,workers
0,Алтайский край,Барнаул,Барнаул,1701000001,53.348056,83.78,1985,NaN,NaN,NaN,...,NaN,6.6,91.0,0.147,NaN,NaN,NaN,NaN,0.213,353.8
1,Алтайский край,Барнаул,Барнаул,1701000001,53.348056,83.78,1990,NaN,NaN,NaN,...,NaN,2.9,99.0,0.208,NaN,NaN,NaN,NaN,0.296,336.7
2,Алтайский край,Барнаул,Барнаул,1701000001,53.348056,83.78,1995,NaN,NaN,NaN,...,NaN,-6.1,107.0,432.600,NaN,NaN,NaN,NaN,440.000,269.5
3,Алтайский край,Барнаул,Барнаул,1701000001,53.348056,83.78,1996,NaN,NaN,NaN,...,NaN,-6.2,107.0,694.600,NaN,NaN,NaN,NaN,756.500,262.8
4,Алтайский край,Барнаул,Барнаул,1701000001,53.348056,83.78,1997,NaN,NaN,NaN,...,NaN,-4.9,108.0,1035.400,NaN,NaN,NaN,NaN,889.900,257.6


In [3]:
themesDf = pd.read_csv('data/themes.csv', sep=',')

In [4]:
campaign = pd.DataFrame({'utm_campaign': [0], 'gender': ['m'], 'age_from': [25], 'age_to': [56], 'city': [''], 'theme': ['']})
cities = regions_grp['settlement'].tolist()
themes = themesDf['theme'].tolist()
for i in range(50):
    ages = [random.randrange(25,55), random.randrange(25,55)]
    campaign.loc[i] = [i, random.choice(['m','f']), min(ages), max(ages), random.choice(cities), random.choice(themes)]
campaign.head()

campaign.to_csv('data/campaign.csv', index=False)

client_id,клиент (рандом клиенты из датасета банка)
product_category_name,продукт (рандом из датасета банка)
utm_source,источник (google, yandex, vk, tenchat, telegram) telegram
utm_content,id объявления (сообщение тг, пост вк и проч.) null
utm_campaign,id кампании (рандом 1-50)
data,дата лида (от 2022-03-01 до 2022-05-31) распределено по 100-300 заявок рандомно на 1 день. всего получается 18000 строк
cpc,стоимость перехода (рандом 15-100)

In [32]:
clientIds = list(set(df['client_id'].tolist()))
products = list(set(df['product_category_name'].tolist()))
dates = pd.date_range(start="2022-03-01",end="2022-05-31")
channelsDf = pd.read_csv('data/telegram_channels.csv', sep=',')
channels = channelsDf['name_id'].tolist()
sources = ['telegram', 'yandex_rsya', 'yandex_dzen', 'partner']
partners = ['banki_ru', 'sravni_ru', 'm_video', 'aliexpress', 'holodilnik_ru', 'megafon']

print(len(clientIds), len(products), len(dates), len(channels))

16797 2 92 1788


In [33]:
leadsCols = ['client_id', 'product_category_name', 'utm_source', 'utm_content', 'utm_campaign', 'date', 'cpc']
leads = pd.DataFrame(columns=leadsCols)

for date in dates:
  actionsPerDate = random.randrange(100,300) 
  for i in range(actionsPerDate):

    source = random.choice(sources)
    channel = None
    if source == 'telegram':
      channel = random.choice(channels)
    elif source == 'partner':
      channel = random.choice(partners)

    values = [
      random.choice(clientIds),
      random.choice(products),
      source,
      channel,
      random.randrange(50),
      date,
      random.randrange(4000,50000)
    ]
    row = dict(zip(leadsCols, values))
    leads.loc[leads.shape[0]] = row

print(leads.shape[0])
leads.head(3)
# themes.to_csv('data/themes.csv', index=False)

18433


,client_id,product_category_name,utm_source,utm_content,utm_campaign,date,cpc
0,1ad6099ac84f192da5032c56f93a61da,Кредитная карта,partner,aliexpress,13,2022-03-01,43513
1,2a561c841ef0a7f4ee0799f6a16b228b,Кредитная карта,telegram,tuzemunio,16,2022-03-01,35550
2,7c966010247d7c62216c55b8dd8288e4,Кредитная карта,partner,banki_ru,37,2022-03-01,31786


In [29]:
leads.to_csv('data/leads.csv', index=False)

In [31]:
channelsDf.to_json(r'data/telegram_channels.json', orient='records', force_ascii=False)
leads.to_json(r'data/leads.json', orient='records', force_ascii=False)